## 对比在相同模型、相同数据下，两种迁移学习的准确率

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision 
import torch.utils.data
import torch.nn.functional as F
import torchvision.transforms as transforms
from torchvision import datasets, models
import os
import time
import copy

# data - hymenoptera_data

In [27]:
data_transforms = {
    "train": transforms.Compose([
        transforms.RandomResizedCrop(224),      # 长宽比裁剪
        transforms.RandomHorizontalFlip(),      #水平翻转
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])
    ]),
    "val": transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
}
data_dir = r"./hymenoptera_data/hymenoptera_data"

image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ["train","val"]}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4,
                                              shuffle=True, num_workers=1)
               for x in ['train', 'val']}

dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

# moudel - resnet 18  

In [30]:
model_18 = models.resnet18(pretrained=True) 

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

# Training function - normal

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

def train_model_N(model,criterion,optimizer,num_epochs=25):
    since = time.time()
    
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    for epoch in range(num_epochs):        # epochs循环
        print("Epoch{}/{}".format(epoch,num_epochs-1))
        print("-"*10)
        
        for phase in ["train","val"]:     # 训练模型还是检测模型
            if phase == "train":
                model.train()
            else:
                model.eval()
                
            running_loss = 0.0
            running_corrects = 0
            
            for inputs, labels in dataloaders[phase]:    # 数据装载位置
#                 inputs = inputs.to(device)
#                 labels = labels.to(device)   # 把数据装载过去
                
                optimizer.zero_grad()
                
                with torch.set_grad_enabled(phase == "train"):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs,1)
                    loss = criterion(outputs,labels)
                    
                    if phase == "train":
                        loss.backward()
                        optimizer.step()
                        
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                
            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]
            
            print("{} Loss:{:.4f} Acc:{:.4f}".format(phase, epoch_loss, epoch_acc))
            
            if phase == "val" and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
                
    time_elapsed = time.time()-since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:.4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model
            


### 训练一：使用模型自带参数训练。

In [15]:
model_18 = models.resnet18(pretrained=True) 

for p,name in zip(model_18.parameters(),model_18.state_dict().keys()):
    p.requires_grad=False
    
model_18.fc = nn.Linear(model_18.fc.in_features,2)

for p in model_18.fc.parameters():
    p.requires_grad = True
    
# for p,name in zip(model_18.parameters(),model_18.state_dict().keys()):
#     print(name,':',p.requires_grad)
    
criterion = nn.CrossEntropyLoss()

optimizer_ft = optim.Adam(model_18.parameters(),weight_decay=0.001)

model_18_train = train_model_N(model_18,criterion,optimizer_ft, num_epochs=5)

Epoch0/4
----------
train Loss:0.6146 Acc:0.6680
val Loss:0.3792 Acc:0.8758
Epoch1/4
----------
train Loss:0.4734 Acc:0.7746
val Loss:0.3268 Acc:0.8562
Epoch2/4
----------
train Loss:0.3758 Acc:0.8238
val Loss:0.2413 Acc:0.9281
Epoch3/4
----------
train Loss:0.3763 Acc:0.8525
val Loss:0.2441 Acc:0.9150
Epoch4/4
----------
train Loss:0.4172 Acc:0.7869
val Loss:0.2171 Acc:0.9281
Training complete in 7m 49s
Best val Acc: 0.9281


### 训练二：用原有模型参数作为初始参数训练模型

In [10]:
model_18 = models.resnet18(pretrained=True) 

model_18.fc = nn.Linear(model_18.fc.in_features, 2)

criterion = nn.CrossEntropyLoss()

optimizer_ft = optim.Adam(model_18.parameters(),weight_decay=0.001)

model_18_train = train_model_N(model_18,criterion,optimizer_ft, num_epochs=5)

Epoch0/4
----------
train Loss:1.0333 Acc:0.5369
val Loss:0.8596 Acc:0.5948
Epoch1/4
----------
train Loss:0.6791 Acc:0.6025
val Loss:0.7359 Acc:0.6863
Epoch2/4
----------
train Loss:0.6739 Acc:0.6352
val Loss:1.2710 Acc:0.4183
Epoch3/4
----------
train Loss:0.7648 Acc:0.5205
val Loss:1.3872 Acc:0.6405
Epoch4/4
----------
train Loss:0.7171 Acc:0.5738
val Loss:0.7802 Acc:0.5752
Training complete in 17m 10s
Best val Acc: 0.6863


# L2正则化：
在optim 中加入参数weight_decay = 0.01 ( 模型自带，只需要调整超参，就可调动）

参考训练1，训练2

# L1 正则化：


In [18]:
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

def train_model_L1(model,criterion,optimizer,num_epochs=25):
    since = time.time()
    
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    for epoch in range(num_epochs):        # epochs循环
        print("Epoch{}/{}".format(epoch,num_epochs-1))
        print("-"*10)
        
        for phase in ["train","val"]:     # 训练模型还是检测模型
            if phase == "train":
                model.train()
            else:
                model.eval()
                
            running_loss = 0.0
            running_corrects = 0
            
            for inputs, labels in dataloaders[phase]:    # 数据装载位置
#                 inputs = inputs.to(device)
#                 labels = labels.to(device)   # 把数据装载过去

                
                optimizer.zero_grad()
                
                with torch.set_grad_enabled(phase == "train"):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs,1)
                    
                    L1 = 0
                    for param in model.parameters():
                        L1 += torch.sum(torch.abs(param))
                        
                    loss = criterion(outputs,labels) + 0.001 * L1
                    
                    
                    if phase == "train":

                        loss.backward()
                        optimizer.step()
                        
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                
            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]
            
            print("{} Loss:{:.4f} Acc:{:.4f}".format(phase, epoch_loss, epoch_acc))
            
            if phase == "val" and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
                
    time_elapsed = time.time()-since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:.4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [19]:
model_18 = models.resnet18(pretrained=True) 

for p,name in zip(model_18.parameters(),model_18.state_dict().keys()):
    p.requires_grad=False
    
model_18.fc = nn.Linear(model_18.fc.in_features,2)

for p in model_18.fc.parameters():
    p.requires_grad = True
    
# for p,name in zip(model_18.parameters(),model_18.state_dict().keys()):
#     print(name,':',p.requires_grad)
    
criterion = nn.CrossEntropyLoss()

optimizer_ft = optim.Adam(model_18.parameters())

model_18_train = train_model_L1(model_18,criterion,optimizer_ft, num_epochs=5)

Epoch0/4
----------
train Loss:168.4039 Acc:0.6885
val Loss:168.1528 Acc:0.8824
Epoch1/4
----------
train Loss:168.2766 Acc:0.7623
val Loss:168.0549 Acc:0.9281
Epoch2/4
----------
train Loss:168.2533 Acc:0.7910
val Loss:168.0428 Acc:0.9216
Epoch3/4
----------
train Loss:168.1949 Acc:0.8033
val Loss:168.0035 Acc:0.9542
Epoch4/4
----------
train Loss:168.1578 Acc:0.8648
val Loss:168.0420 Acc:0.9020
Training complete in 8m 21s
Best val Acc: 0.9542


### 结论：

由于组成图片的基础都是相似的，所以在大厂训练好的模型中，其自带的参数具有很好的参考意义。那么在本身不具有特别好的硬件条件下，建议还是不要更新模型自带参数，冻结模型的大部分层次，对于小数据会有更好的效果。

由于使用的是Adam优化器，学习率本身会自我更新，在冻结模型自带参数的条件下，L1能更有效的调节最后全连接层的关键参数，使得模型效果比前两个的实验效果更好。